# DM - project (based on facenet paper)
by:
i170093 - Shayaan Farooq

i170305 - Muhammad Huzaifa

i170317 - Muhammad Bilal

Method:

We have trained a 3-path siamese network, with offline triplet mining.
The classifier we have utilised is KNN
The model utilises the same network (zfnet) explained in the paper
Specifications are as follows:
* LFW dataset, where 10 random classes (with 20+ images) are split off for test data
* the images are centre cropped to reduce their size to 160x160
* epochs = 10
* margin = 0.2

In [1]:
import torch
import torchvision
from torch.utils.data import  Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import combinations,permutations
# from scipy.spatial.distance import cdist
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances
import time
import math
# from google.colab import files
# from torchvision.models import resnet34
import torchvision.models as models
from PIL import Image
# from torchvision.models import resnet152
import random

In [2]:
class KNearestNeighbor(object):
  """ a kNN classifier with L2 distance """

  def __init__(self):
    pass

  def train(self, X, y):
    self.X_train = np.asarray(X)
    self.y_train = np.asarray(y)
    
  def predict(self, X, k=1, num_loops=0):
    if num_loops == 0:
      dists = self.compute_distances_no_loops(X)
    elif num_loops == 1:
      dists = self.compute_distances_one_loop(X)
    elif num_loops == 2:
      dists = self.compute_distances_two_loops(X)
    else:
      raise ValueError('Invalid value %d for num_loops' % num_loops)

    return self.predict_labels(dists, k=k)

  def compute_distances_two_loops(self, X):
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train))
    for i in range(num_test):
      for j in range(num_train):
        dists[i][j] = np.sqrt(np.sum((X[i] - self.X_train[j]) ** 2))
    return dists

  def compute_distances_one_loop(self, X):
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train))
    for i in range(num_test):
      dists[i] = np.sqrt(np.sum((self.X_train - X[i]) ** 2, axis=1))
    return dists

  def compute_distances_no_loops(self, X):
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train)) 
    s1 = np.sum(X ** 2, axis=1)
    s2 = np.sum(self.X_train ** 2, axis=1)
    s = s1.reshape((num_test, 1)) + s2 - 2 * X.dot(self.X_train.T)
    dists = np.sqrt(s)
    return dists

  def predict_labels(self, dists, k=1):
    num_test = dists.shape[0]
    y_pred = np.zeros(num_test)
    for i in range(num_test):
      # A list of length k storing the labels of the k nearest neighbors to
      # the ith test point.
      closest_y = []

      knn_ix = dists[i].argsort()[:k]
      closest_y = self.y_train[knn_ix]

      values, counts = np.unique(closest_y, return_counts=True)
      y_pred[i] = values[counts == counts.max()].min()

    return y_pred


In [3]:
class testnet(nn.Module):
    def __init__(self, embedding_size, pretrained=False):
        super (testnet, self).__init__()
        self.conv1 = nn.Conv2d(3,64, 7, stride=2)
        self.pool1 = nn.MaxPool2d(3, stride=2, padding = 1)
        self.rnorm1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv2a = nn.Conv2d(64,64, 1, stride=1)
        self.conv2 = nn.Conv2d(64,192, 3, stride=1)
        self.rnorm2 = nn.BatchNorm2d(192)
        self.pool2 = nn.MaxPool2d(3, stride=2, padding = 1)
        self.relu2 = nn.ReLU()

        self.conv3a = nn.Conv2d(192,192, 1, stride=1)
        self.conv3 = nn.Conv2d(192,384, 3, stride=1)
        self.pool3 = nn.MaxPool2d(3, stride=2,padding = 1)
        self.relu3 = nn.ReLU()

        self.conv4a = nn.Conv2d(384,384, 1, stride=1)
        self.conv4 = nn.Conv2d(384,256, 3, stride=1, padding = 1)

        self.conv5a = nn.Conv2d(256,256, 1, stride=1)
        self.conv5 = nn.Conv2d(256,256, 3, stride=1, padding = 1)

        self.conv6a = nn.Conv2d(256,256, 1, stride=1)
        self.conv6 = nn.Conv2d(256,256, 3, stride=1, padding = 1)

        self.pool4 = nn.MaxPool2d(3, stride=2, padding = 1)
        #concatinating
#         2304
        self.fc1 = nn.Linear(6400, 128*32*1)#what is maxout
        self.fc2 = nn.Linear(128*32*1, 128*32*1)#what is maxout
        self.fc7128 = nn.Linear(128*32*1, 128*1*1)#what is maxout
        #l2 normalization

    def l2_norm(self, input):
        input_size = input.size()
        buffer     = torch.pow(input, 2)
        normp      = torch.sum(buffer, 1).add_(1e-10)
        norm       = torch.sqrt(normp)
        _output    = torch.div(input, norm.view(-1, 1).expand_as(input))
        output     = _output.view(input_size)

        return output

    def forward_once(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.rnorm1(x)
        x = self.relu1(x)

        x = self.conv2a(x)
        x = self.conv2(x)
        x = self.rnorm2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = self.conv3a(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.relu3(x)

        x = self.conv4a(x)
        x = self.conv4(x)
        x = self.conv5a(x)
        x = self.conv5(x)
        x = self.conv6a(x)
        x = self.conv6(x)
        x = self.pool4(x)

        #concatinating
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc7128(x)
        #l2 normalization
        x = self.l2_norm(x)
        return x

    def forward(self, anchor, positive, negative):
#         for triplet
        a = self.forward_once(anchor)
        p = self.forward_once(positive)
        n = self.forward_once(negative)

        return a, p, n

class OfflineTripletLoss(nn.Module):
    def __init__(self, margin):
        super(OfflineTripletLoss, self).__init__()
        self.margin = margin
    def forward(self, a, p , n):
#         do something here to improve convergance, maybe
# check distances for hard and semi hard triplets, turn easy triplet distance to maybe 0?
        distance_positive = (a - p).pow(2).sum(1)
        distance_negative = (a - n).pow(2).sum(1)
        temp = distance_positive - distance_negative + self.margin
#         print (temp)
        losses = F.relu(temp)
        return losses.mean()

In [4]:

#LFW dataset inheriting 'Dataset'
class LFWDataset(Dataset):
    
    
    def __init__(self, csv_file, root_dir, testFoldersRequired, imageThreshold, transform):
        #we gotta data load
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.annotations = pd.read_csv(csv_file) #contains csv read data    
        self.root_dir = root_dir
        self.transform = transform
        self.testFoldersRequired=testFoldersRequired,
        self.imageThreshold= imageThreshold,
        self.trainImagePaths=[] #contains path that get_item uses to return image tensor
        self.trainImageLabels=[]; #corresponding to above paths, this contains labels, also returned in get_item
        
        self.testImagePaths=[] #retrived using getTestImagePaths function and used in other custom dataset for testing
        self.testImageLabels= [] #corresponding labels

        self.triplets= []
        self.testFolders=[] #folder numbers correspond to rows in csv
        self.testFolderCount=int(0)
        self.annotationLength= len(self.annotations)
        self.trainFolders=[] #all other folders than test
        
        
        #class separator
        #random so that each time we train/separate classes its different
        for i in random.sample(range(self.annotationLength), self.annotationLength):
                totalImagesForLabel= int(self.annotations.iloc[i, 1])
                
                #if testfolders are less than 10, and images in current folder are more than 10, SEPARATE IT
                if(self.testFolderCount < self.testFoldersRequired[0] and totalImagesForLabel >self.imageThreshold[0]): #[0] is done cuz its tuple idk why but it works
                    self.testFolders.append(i) #adds csv file row number to testFolder
                    self.testFolderCount+=1
                else:
                    self.trainFolders.append(i) # add folder to training 
        
        #main loop, loops each person in train Folders
        for i in self.trainFolders:
            
            totalImagesForLabel= int(self.annotations.iloc[i, 1])
            #this loop generates path for each image for a person
            #if condition decides if it needs to be added to trainset or testset
            for j in range(totalImagesForLabel):  
                #All are sent to train
                self.trainImageLabels.append(i)
                self.trainImagePaths.append(root_dir 
                                     + "/" 
                                     + self.annotations.iloc[i, 0] 
                                     + "/" 
                                     + self.annotations.iloc[i, 0] 
                                     + '_' 
                                     + "%0004d" % (j+1) + ".jpg")

                #generating triplet
                anchor=len(self.trainImageLabels) - 1 #index in trainImages
                    
                if(totalImagesForLabel<5):
                    self.generateTriplet(anchor,j, totalImagesForLabel) #j is position in folder
                    
                elif(totalImagesForLabel<10):
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    
                elif(totalImagesForLabel<15):
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    
                elif(totalImagesForLabel<20):
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    
                else:
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)
                    self.generateTriplet(anchor,j, totalImagesForLabel)

                    
                    
                
                
        #UNCOMMENT TO SEE SPLIT NUMBERS WITH CURRENT CONFIGS IN if elif elifs ABOVE
        print("No of Train Images: " + str(len(self.trainImagePaths)))
        print("No of Triplets: " + str(len(self.triplets)))
        
    def __len__(self):
        #does this: len(dataset for training)
        return len(self.triplets)

    #this will return image, label from the train set
    def __getitem__(self, idx):
        triplet= self.triplets[idx]
        
        anc_img_path= self.trainImagePaths[triplet[0]]
        anc_image=Image.open(anc_img_path)
#         anc_image= anc_image.crop((73, 91, 173, 191))
        anc_image= anc_image.crop((43, 61, 203, 221))

        label_for_image= self.trainImageLabels[triplet[0]] # number eg. 15th row in csv file 
            
        pos_img_path= self.trainImagePaths[triplet[1]]
        pos_image=Image.open(pos_img_path)
        pos_image= pos_image.crop((43, 61, 203, 221))
        
        neg_img_path= self.trainImagePaths[triplet[2]]
        neg_image=Image.open(neg_img_path)
        neg_image= neg_image.crop((43, 61, 203, 221))
        
        if self.transform:
            anc_image = self.transform(anc_image)
            pos_image = self.transform(pos_image)
            neg_image = self.transform(neg_image)

        return (anc_image, pos_image, neg_image, label_for_image)

    def generateTriplet(self, anchor, j, totalImagesForLabel):
        triplet=[]
        triplet.append(anchor) #anchor appended
        triplet.append(self.generatePositive(anchor, j , totalImagesForLabel)) #j is achors number in folder
        triplet.append(self.generateNegative(anchor-j, totalImagesForLabel))
        self.triplets.append(triplet)
            
        
    
    def generatePositive(self,a,positionInFolder, noOfImages):
        if(noOfImages==1):
            return a
        
        rangeForRandom= list(range(0,positionInFolder)) + list(range(positionInFolder+1, noOfImages))
        randomNo=random.choice(rangeForRandom)
        if(randomNo>positionInFolder):
            return a+(randomNo - positionInFolder) #position that its going to have in imagePathArray
        else:
            return a-(positionInFolder - randomNo)
        
    def generateNegative(self, startIndexOfLabel, noOfImagesForLabel):
        rangeForRandom= list(range(0,startIndexOfLabel)) + list(range(startIndexOfLabel+noOfImagesForLabel+1, 12817))
        return random.choice(rangeForRandom)
        
    def getTriplets(self):
        return self.triplets
    
    def getTestFolders(self):
        return self.testFolders
    
    def getTrainImagePaths(self):
        return self.trainImagePaths
        
    def getTrainImageLabels(self):
        return self.trainImageLabels
    
    def getTestImagePaths(self):
        return self.testImagePaths

    def getTestImageLabels(self):
        return self.testImageLabels
    
    def getNameForLabel(self,i):
        return self.annotations.iloc[i,0] #gets 0, returns AJ_Cook or whatever first persons name is innit


In [5]:
class separatedClassesDataset(Dataset):
    
    
    def __init__(self, csv_file, root_dir, testFolders, noOfImagesToKeepForTestPerFolder, transform):
        #we gotta data load
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            testImageP(list of strings): contains paths to images separated for training
            testImageL(list of ints): contains labels to above images
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.annotations = pd.read_csv(csv_file) #contains csv read data
        self.root_dir = root_dir
        self.transform = transform
        
        self.testFolders=testFolders
        self.noOfImagesToKeepForTestPerFolder= noOfImagesToKeepForTestPerFolder 

        #this will contain paths to those images jinki embedddings niklayn gi for knn
        self.forwardPassImagePaths=[]
        self.forwardPassImageLabels=[]
        
        #this will contain paths to images that we gotta test after embedding+knn
        self.testImagePaths=[] 
        self.testImageLabels=[]
        self.noOfImagesToForwardPerFolder = 0
        
        for i in testFolders:
            numAddedToForward=0
            totalImagesForLabel= int(self.annotations.iloc[i, 1])
            
            self.noOfImagesToForwardPerFolder = totalImagesForLabel - self.noOfImagesToKeepForTestPerFolder
            
            for j in random.sample(range(totalImagesForLabel), totalImagesForLabel):
                    if(numAddedToForward<self.noOfImagesToForwardPerFolder):
                        self.forwardPassImageLabels.append(i)
                        self.forwardPassImagePaths.append(root_dir 
                                         + "/" 
                                         + self.annotations.iloc[i, 0] 
                                         + "/" 
                                         + self.annotations.iloc[i, 0] 
                                         + '_' 
                                         + "%0004d" % (j+1) + ".jpg")
                        
                        numAddedToForward+=1
                        
                    else:
                        self.testImageLabels.append(i)
                        self.testImagePaths.append(root_dir 
                                         + "/" 
                                         + self.annotations.iloc[i, 0] 
                                         + "/" 
                                         + self.annotations.iloc[i, 0] 
                                         + '_' 
                                         + "%0004d" % (j+1) + ".jpg")
                        
            
        
    def __len__(self):
        
        return len(self.forwardPassImagePaths)

    #this will return image, label
    def __getitem__(self, idx):
        img_path= self.forwardPassImagePaths[idx]
        image=Image.open(img_path)
        image= image.crop((43, 61, 203, 221))
#         (73, 91, 173, 191)
        label_for_image= self.forwardPassImageLabels[idx]
        
        if self.transform:
            image = self.transform(image)

        return (image, label_for_image)

    def getTestImagePaths(self):
        return self.testImagePaths
    
    def getTestImageLabels(self):
        return self.testImageLabels
    
    def getNameForLabel(self,i):
        return self.annotations.iloc[i,0] #gets 0, returns AJ_Cook or whatever first persons name is innit


In [6]:
class LFWTestDataset(Dataset):
    
    
    def __init__(self, csv_file, root_dir, testImagePaths, testImageLabels, transform):
        #we gotta data load
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            testImageP(list of strings): contains paths to images separated for training
            testImageL(list of ints): contains labels to above images
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.annotations = pd.read_csv(csv_file) #contains csv read data
        self.root_dir = root_dir
        self.transform = transform
        
        self.testImagePaths=testImagePaths
        self.testImageLabels= testImageLabels 
        
        
        
        
    def __len__(self):
        #does this: len(dataset for testing)
        return len(self.testImagePaths)

    #this will return image, label from the test set
    def __getitem__(self, idx):
        img_path= self.testImagePaths[idx]
        image=Image.open(img_path)
        #print("precrop size: ")
        #print(image.size)
        image= image.crop((43, 61, 203, 221))
#         (73, 91, 173, 191)
        #print("postcrop size: ")
        #print(image.size)
        label_for_image= self.testImageLabels[idx] # number eg. 15th row in csv file 

        if self.transform:
            image = self.transform(image)

        return (image, label_for_image)

    def getNameForLabel(self,i):
        return self.annotations.iloc[i,0] #gets 0, returns AJ_Cook or whatever first persons name is innit


In [7]:
class LFWTrainDataset(Dataset):
    
    
    def __init__(self, csv_file, root_dir, trainImagePaths, trainImageLabels, transform):
        #we gotta data load
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            testImageP(list of strings): contains paths to images separated for training
            testImageL(list of ints): contains labels to above images
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.annotations = pd.read_csv(csv_file) #contains csv read data
        self.root_dir = root_dir
        self.transform = transform
        
        self.trainImagePaths=trainImagePaths
        self.trainImageLabels= trainImageLabels 

        
    def __len__(self):
        #does this: len(dataset for testing)
        return len(self.trainImagePaths)

    #this will return image, label from the test set
    def __getitem__(self, idx):
        img_path= self.trainImagePaths[idx]
        image=Image.open(img_path)
        #print("precrop size: ")
        #print(image.size)
#         image= image.crop((73, 91, 173, 191))
        image= image.crop((43, 61, 203, 221))
#         print("postcrop size: ")
#         print(image.size)
        label_for_image= self.trainImageLabels[idx] # number eg. 15th row in csv file 

        if self.transform:
            image = self.transform(image)

        return (image, label_for_image)

    def getNameForLabel(self,i):
        return self.annotations.iloc[i,0] #gets 0, returns AJ_Cook or whatever first persons name is innit


In [31]:
#Set Paths
root= '/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
csvFile= '/kaggle/input/lfw-dataset/lfw_allnames.csv'


#==================for training this dawg=================================  
trainDataset= LFWDataset(csv_file= csvFile, 
                      root_dir= root,
                      testFoldersRequired=10 ,
                      imageThreshold=20 ,
                      transform= transforms.ToTensor())

print("Test Data Classes: ")
print(trainDataset.annotations.iloc[trainDataset.testFolders])




#singlke image train dataset FOR KNNNNNNN
singleImageTrainDataset = LFWTrainDataset(csv_file= csvFile, 
                      root_dir= root,
                      trainImagePaths= trainDataset.getTrainImagePaths(),
                      trainImageLabels= trainDataset.getTrainImageLabels(),      
                      transform= transforms.ToTensor())



#==================for embeddings then knn=================================  
testFolders=trainDataset.getTestFolders()
forwardPassDataset = separatedClassesDataset(root_dir= root,
                                             csv_file= csvFile,
                                             testFolders=testFolders,
                                             noOfImagesToKeepForTestPerFolder= 10,
                                             transform= transforms.ToTensor()
                                             ) 
# print("\n-----------Forward Pass Paths------------------------")
# for i in range(len(forwardPassDataset.forwardPassImagePaths)):
#     print(forwardPassDataset.forwardPassImagePaths[i])



#==================for prediction=================================
testImagePaths=forwardPassDataset.getTestImagePaths() #[rock1.jpg, rock2.jpg, AJ_Cook.jpg]
testImageLabels=forwardPassDataset.getTestImageLabels() #[233, 233, 0] -> corresponds to csv/ folder numbers

testDataset = LFWTestDataset(root_dir= root,
                             csv_file= csvFile,
                             testImagePaths=testImagePaths,
                             testImageLabels= testImageLabels,
                             transform= transforms.ToTensor()
                             ) 

# print("\n-----------Test Paths-----------------")
# for i in range(len(testDataset.testImagePaths)):
#     print(testDataset.testImagePaths[i])



No of Train Images: 12854
No of Triplets: 28265
Test Data Classes: 
                   name  images
223     Amelie_Mauresmo      21
4333       Pete_Sampras      22
4836   Rudolph_Giuliani      26
1892  Gerhard_Schroeder     109
786        Carlos_Menem      21
3434      Mahmoud_Abbas      29
373        Ariel_Sharon      77
1207      David_Beckham      31
2514     Jennifer_Lopez      21
3327  Lindsay_Davenport      22


In [32]:
tripletTrainLoader= torch.utils.data.DataLoader(trainDataset, batch_size=32, shuffle=True, num_workers=2)
singleTrainLoader= torch.utils.data.DataLoader(singleImageTrainDataset, batch_size=32, shuffle=True, num_workers=2)
forwardTestLoader= torch.utils.data.DataLoader(forwardPassDataset, batch_size=32, shuffle=True, num_workers=2)
singleTestLoader= torch.utils.data.DataLoader(testDataset, batch_size=32, shuffle=True, num_workers=2)

In [33]:
NUM_EPOCHS = 10
margin = 0.2

Running this training cell just top show output of loss, actually trained/saved model is loaded in next cell 

In [34]:
# ---------------------training model for offline triplets/siamese network------------------------------
model = testnet(128, False)
model = model.cuda()

# optimizer = optim.Adam(model.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

loss_func = OfflineTripletLoss(margin)
loss_func = loss_func.cuda()


for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    for i,data in enumerate(tripletTrainLoader):#this loader returns triplets from train dataset
# will get 4 returns values instead of 2
        anchors, positives, negatives, labels = data
    
        anchors = anchors.cuda()
        positives = positives.cuda()
        negatives = negatives.cuda()
        # labels = labels.cuda()

        aVectors,pVectors,nVectors = model(anchors, positives, negatives)
        optimizer.zero_grad()

# loss function does not take in labels
        loss = loss_func(aVectors, pVectors , nVectors)
        loss.backward()
        optimizer.step()
    print("--- %s seconds ---" % (time.time() - start_time))
    print('Epoch:{}, Loss:{}'.format(epoch+1, float(loss)))


--- 236.28417682647705 seconds ---
Epoch:1, Loss:0.059837330132722855
--- 227.43655848503113 seconds ---
Epoch:2, Loss:0.057087793946266174
--- 228.86970448493958 seconds ---
Epoch:3, Loss:0.05640449747443199
--- 237.65939712524414 seconds ---
Epoch:4, Loss:0.03484893590211868
--- 239.0373694896698 seconds ---
Epoch:5, Loss:0.012365564703941345
--- 236.2841441631317 seconds ---
Epoch:6, Loss:0.0
--- 237.3864140510559 seconds ---
Epoch:7, Loss:0.0
--- 233.12666273117065 seconds ---
Epoch:8, Loss:0.0
--- 233.63143372535706 seconds ---
Epoch:9, Loss:0.0
--- 240.45462608337402 seconds ---
Epoch:10, Loss:0.0


In [36]:
# loading saved model parameters into a variable
state_dict = torch.load('/kaggle/input/trained-models/zfnet_10epoch_offline_32batch_160imsize.pth')
# print(state_dict.keys())

# updating model parameters with loaded variable
model = testnet(128,  False)
model.load_state_dict(state_dict)
model = model.cuda()


In [35]:
# --------------creating training dps for final knn train---------------
final_embeddings = torch.empty(0).float()
final_labels = torch.empty(0)
for i, data in enumerate(singleTrainLoader):#this loader return single train images
#     start_time = time.time()
    imgs, labels = data
    imgs = imgs.cuda()
    
    vectors = model.forward_once(imgs)
    
    final_embeddings = torch.cat((final_embeddings,vectors.detach().cpu()),0)
    final_labels = torch.cat((final_labels,labels.float()),0)
    
#     print("--- %s seconds ---" % (time.time() - start_time))
print (final_embeddings.size())

torch.Size([12854, 128])


In [37]:
#-----------------creating forwarding dps from test data for knn training--------------
forward_embeddings = torch.empty(0).float()
forward_labels = torch.empty(0)
for i, data in enumerate(forwardTestLoader):#this loader return single test images
#     start_time = time.time()
    forwardimgs, forwardlabels = data
    forwardimgs = forwardimgs.cuda()
    
    forwardvectors = model.forward_once(forwardimgs)
    
    forward_embeddings = torch.cat((forward_embeddings,forwardvectors.detach().cpu()),0)
    forward_labels = torch.cat((forward_labels,forwardlabels.float()),0)
    
#     print("--- %s seconds ---" % (time.time() - start_time))
print (forward_embeddings.size())

torch.Size([279, 128])


In [38]:
#-----------------creating testing dps for knn classification--------------
test_embeddings = torch.empty(0).float()
test_labels = torch.empty(0)
for i, data in enumerate(singleTestLoader):#this loader return single test images
#     start_time = time.time()
    testimgs, testlabels = data
    testimgs = testimgs.cuda()
    
    testvectors = model.forward_once(testimgs)
    
    test_embeddings = torch.cat((test_embeddings,testvectors.detach().cpu()),0)
    test_labels = torch.cat((test_labels,testlabels.float()),0)
    
#     print("--- %s seconds ---" % (time.time() - start_time))
print (test_embeddings.size())

torch.Size([100, 128])


In [ ]:
# train accuracy

knn = KNearestNeighbor()

knn.train(final_embeddings.numpy(),final_labels.numpy())
ans = knn.predict(final_embeddings.detach().numpy(),1,0)
# print (ans[10:30])
print (ans)
print (final_labels)
count = 0.0
for i in range(len(final_labels)):
    if(ans[i] == final_labels[i]):
        count += 1
print ("accuracy is: " + str(count/len(final_labels)))

In [ ]:
# # combining existing embeddings and forward embeddings
# X = torch.cat((final_embeddings,forward_embeddings),0)
# Y = torch.cat((final_labels,forward_labels),0)

In [39]:
# revised test accuracy
knn = KNearestNeighbor()

knn.train(forward_embeddings.detach().numpy(),forward_labels.detach().numpy())
ans = knn.predict(test_embeddings.detach().numpy(),3,0)
# ans = knn.predict(final_embeddings.detach().numpy(),1,0)
# print (ans[10:30])
print (ans)
print (test_labels)
count = 0.0
for i in range(len(ans)):
    if(ans[i] == test_labels[i]):
        count += 1
print ("accuracy is: " + str(count/len(test_labels)))

[ 786. 4836.  223. 4836.  373.  223.  223. 1892. 4836. 1892.  223. 3327.
  786. 2514. 1207. 2514. 4836. 3434.  786.  373. 1892.  223.  373. 4333.
 1892.  373. 2514. 1207. 4333. 3327. 1207. 3327. 1892. 2514.  373.  223.
 3434. 1892. 3434. 3434. 3434. 4836. 3327.  373.  223.  786. 4836. 3327.
 1892.  373.  786. 1207. 4333.  223. 4333. 4333. 3434. 3434.  786. 3327.
 4836. 4836. 1207. 1892. 3327. 1892. 1207.  786. 1207.  786.  786. 1207.
 4836.  786. 2514. 2514. 2514. 1207. 4333.  786.  373.  223. 3434.  223.
 2514. 4333.  373. 2514. 3434. 2514. 4333. 1892.  786.  373. 1892. 1207.
 4333. 3327. 1207. 1207.]
tensor([ 786., 4836.,  223., 4836.,  373.,  223., 3327., 1892., 4836., 1892.,
         223., 3327.,  786., 2514., 1207., 2514., 4836., 3434.,  786.,  373.,
        1892.,  223.,  373., 4333., 1892.,  373., 2514., 1207., 4333., 3327.,
        3327., 3327., 1892., 2514.,  373.,  223., 3434., 1892., 3434., 3434.,
        3434., 4836., 3327.,  373.,  223.,  786., 4836., 3327., 1892.,  373.,


In [ ]:
# saving the mdoel and downloading
torch.save(model.state_dict(), 'zfnet_10epoch_offline_500batch_160imsize.pth')

# download checkpoint file
# files.download('embeddingnet_5epoch_bathchard.pth')